<a href="https://colab.research.google.com/github/JimKing100/nfl-test/blob/master/neural-network/Article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import numpy as np
import pandas as pd

from sklearn import preprocessing

from pandas import DataFrame
from pandas import Series
from pandas import concat

from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from math import sqrt
from matplotlib import pyplot

import numpy

Using TensorFlow backend.


In [2]:
# Import data
players_df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/nfl-fantasy-ds/master/data/raw/players_full.csv')
offense_pred_df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/nfl-fantasy-ds/master/data/actuals/actuals_offense.csv')
kicker_pred_df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/nfl-fantasy-ds/master/data/actuals/actuals_kickers.csv')
kickers_df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/nfl-fantasy-ds/master/data/raw/rookies_non_kicker.csv')
offense_df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/nfl-fantasy-ds/master/data/raw/rookies_non_offense.csv')
bye_df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/nfl-fantasy-ds/master/data/raw/bye.csv')
injury_df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/nfl-fantasy-ds/master/data/raw/injury.csv')
player_df = pd.concat([kickers_df, offense_df], ignore_index=True)
original_df = pd.concat([offense_pred_df, kicker_pred_df], ignore_index=True)

In [3]:
# The dataframe of actual offensive points for each game from 2000-2019
original_df.head()

,player,first,last,name,position1,2000-game1,2000-game2,2000-game3,2000-game4,2000-game5,2000-game6,2000-game7,2000-game8,2000-game9,2000-game10,2000-game11,2000-game12,2000-game13,2000-game14,2000-game15,2000-game16,2001-game1,2001-game2,2001-game3,2001-game4,2001-game5,2001-game6,2001-game7,2001-game8,2001-game9,2001-game10,2001-game11,2001-game12,2001-game13,2001-game14,2001-game15,2001-game16,2002-game1,2002-game2,2002-game3,...,2017-game9,2017-game10,2017-game11,2017-game12,2017-game13,2017-game14,2017-game15,2017-game16,2018-game1,2018-game2,2018-game3,2018-game4,2018-game5,2018-game6,2018-game7,2018-game8,2018-game9,2018-game10,2018-game11,2018-game12,2018-game13,2018-game14,2018-game15,2018-game16,2019-game1,2019-game2,2019-game3,2019-game4,2019-game5,2019-game6,2019-game7,2019-game8,2019-game9,2019-game10,2019-game11,2019-game12,2019-game13,2019-game14,2019-game15,2019-game16
0,AD-0100,Andy,Dalton,Andy Dalton,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.6,18.56,19.16,19.26,7.64,0.52,11.42,21.68,16.52,26.60,14.08,24.78,12.92,17.16,7.92,20.20,6.92,19.34,8.10,0.00,0.00,0.00,0.00,0.00,21.72,19.64,16.06,3.64,19.98,13.60,18.34,17.16,0.00,0.00,0.00,13.32,9.58,2.14,33.84,16.60
1,AM-1150,A.J.,McCarron,A.J. McCarron,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,1.88,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.32,0.00,0.00,-0.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.90
2,AR-1300,Aaron,Rodgers,Aaron Rodgers,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,21.90,0.00,0.00,25.94,16.04,20.90,15.02,28.68,28.40,16.94,19.16,18.06,22.58,11.92,15.32,20.24,13.46,43.88,1.04,12.92,14.36,14.30,25.48,9.42,18.32,44.76,28.10,12.94,10.02,9.46,28.12,11.40,14.42,9.34,19.02
3,AT-0160,Alex,Tanney,Alex Tanney,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00
4,BA-0375,Brandon,Allen,Brandon Allen,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.02,13.40,2.08,0.00,0.00,0.00,0.00,0.00


In [4]:
# The dataframe of all 2019 offensive players (kickers and offense)
player_df.head()

,player,first,last,name,position1,start
0,AV-0400,Adam,Vinatieri,Adam Vinatieri,K,1996
1,MB-4600,Matt,Bryant,Matt Bryant,K,2002
2,RG-1500,Robbie,Gould,Robbie Gould,K,2005
3,MN-0800,Mike,Nugent,Mike Nugent,K,2005
4,SG-0800,Stephen,Gostkowski,Stephen Gostkowski,K,2006


In [5]:
# The dataframe of bye weeks for 2019 teams
bye_df.head()

,player,first,last,name,position1,bye-week
0,SF,San Francisco,49ers,San Francisco 49ers,DF,4
1,CHI,Chicago,Bears,Chicago Bears,DF,6
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,9
3,BUF,Buffalo,Bills,Buffalo Bills,DF,6
4,DEN,Denver,Broncos,Denver Broncos,DF,10


In [6]:
# The dataframe of injuries for 2019 players
injury_df.head()

,uid,gid,year,week,player,first,last,name,position1,team,details,pstat,gstat
0,29844,5194,2019,10,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
1,30530,5213,2019,11,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
2,31097,5228,2019,12,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
3,31629,5242,2019,13,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
4,32187,5256,2019,14,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR


In [7]:
# Frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

In [8]:
# Scale train and test data to [-1, 1]
def scale(train, test):
    # Fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # Transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # Transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

In [9]:
# Inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = numpy.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [10]:
# Fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    print('fit_lstm function: X = ', X)
    print('                   y = ', y)
    X = X.reshape(X.shape[0], 1, X.shape[1])
    print('                   X (reshaped) = ', X)
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
      model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
      model.reset_states()
    return model

In [11]:
# Make a one-step forecast
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]

In [12]:
# Create the time series for a player
def make_series(p, c):
    column_names = ['game', 'points']
    df = pd.DataFrame(columns = column_names)

    for i in range (c, 321):
        game = i
        col = i + 4
        points = original_df.iloc[:, col].loc[(original_df['player']==p)].iloc[0]
        df = df.append({'game': game, 'points': points}, ignore_index=True)

    s = pd.Series(df['points'].values, index=df['game'])
    return s

In [13]:
def lstm_pred(p, np, c):
    series = make_series(p, c)
    X = series.values
    supervised = timeseries_to_supervised(X, 1)
    supervised_values = supervised.values

    print('lstm_pred function: series = ', series)
    print('                    X = ', X)
    print('                    supervised = ', supervised)
    print('                    supervised_values = ', supervised_values)

    # Split data into train and test-sets
    train, test = supervised_values[0:-np], supervised_values[-np:]
    print('                    train = ', train)
    print('                    test = ', test)

    # Transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    print('                    scaler = ', scaler)
    print('                    train_scaled = ', train_scaled)
    print('                    test_scaled = ', test_scaled)

    # Fit the model
    lstm_model = fit_lstm(train_scaled, 1, 100, 1)

    # Forecast the entire training dataset to build up state for forecasting
    train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
    print('lstm_predict function: train_reshaped = ', train_reshaped)
    lstm_model.predict(train_reshaped, batch_size=1)

    # Walk-forward validation on the test data
    yhat_sum = 0
    for i in range(len(test_scaled)):
        # Make one-step forecast
        X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
        yhat = forecast_lstm(lstm_model, 1, X)
        # Invert scaling
        yhat = invert_scale(scaler, X, yhat)
        print('                       yhat = ', yhat)
        # Sum the weekly forecasts
        yhat_sum = yhat_sum + yhat

    print('                       yhat_sum = ', yhat_sum)

    return yhat_sum

In [14]:
# Add a row to the final_df dataframe
# Each row represents the predicted points for each player
def add_row(df, p, f, l, n, pos, wc, wp, wa, cur, pred, act):

    df = df.append({'player': p,
                    'first': f,
                    'last': l,
                    'name': n,
                    'position': pos,
                    wc: cur,
                    wp: pred,
                    wa: act
                    }, ignore_index=True)
    
    return df

In [15]:
# The main code for iterating through the player(offense and kicker) list, calculating the points and adding the rows
# to the final_df dataframe.
def main(w):
    
    # Create the column names
    week_cname = 'week' + str(w) + '-cur'
    week_pname = 'week' + str(w) + '-pred'
    week_aname = 'week' + str(w) + '-act'

    column_names = ['player',
                    'first',
                    'last',
                    'name',
                    'position',
                    week_cname, 
                    week_pname,
                    week_aname
                     ]

    p_df = player_df.loc[player_df['name'] == 'Aaron Rodgers']
    player_list = p_df['player'].tolist()

    # The column is the column of the dataframe (309 = 2019-game1)
    end_col = 308 + w

    week_df = pd.DataFrame(columns = column_names)

    for player in player_list:
        print(player)

        first = player_df['first'].loc[(player_df['player']==player)].iloc[0]
        last = player_df['last'].loc[(player_df['player']==player)].iloc[0]
        name = player_df['name'].loc[(player_df['player']==player)].iloc[0]
        position1 = player_df['position1'].loc[(player_df['player']==player)].iloc[0]
        start_year = player_df['start'].loc[(player_df['player']==player)].iloc[0]
        row = original_df.index[(original_df['player']==player)][0]
        team = players_df['cteam'].loc[(players_df['player']==player)].iloc[0]

        # Calculate the injury week
        injury_weeks = injury_df['week'].loc[(injury_df['player']==player)]
        if (len(injury_weeks)==0):
            injury_week = 18
        else:
            injury_week = min(injury_weeks)

        bye_week = bye_df['bye-week'].loc[(bye_df['player']==team)].iloc[0]

        # Handle players who starte prior to 2000
        if start_year < 2000:
            start_year = 2000
        col = ((start_year - 2000) * 16) + 5

        # Handle the bye week
        if w > bye_week:
            new_col = end_col - 1
        else:
            new_col = end_col

        train_data = original_df.iloc[row, col:new_col]

        # Calculate current points
        if w == 1:
            current = original_df.iloc[row, 293:309]
        else:  
            current = original_df.iloc[row, 309:new_col]
        cur_points = current.sum()

        # Calculate actual points
        actuals = original_df.iloc[row, new_col:325]
        act_points = actuals.sum()

        # Run the model excluding players without enough data
        if (start_year < 2016) & (train_data.sum() > 50):
        
            if w > bye_week:
                n_periods = 17 - w + 1
            else:
                n_periods = 17 - w

            if (injury_week <= w):
                pred_points = 0
            else:
                # Calculate predicted points
                print('main function: player = ', player)
                print('               n_periods = ', n_periods)
                print('               col = ', col)
                pred_points = lstm_pred(player, n_periods, col)
        else:
            pred_points = train_data.mean() * (18 - w)

        week_df = add_row(week_df, player, first, last, name, position1, week_cname,
                          week_pname, week_aname, cur_points, pred_points, act_points)
    return week_df

In [16]:
# Run for seasons 1 through 17
for i in range(1, 2):
    week = i
    final_df = main(week)

    week_pred = 'week' + str(week) + '-pred'
    week_act = 'week' + str(week) + '-act'
    week_diff = 'week' + str(week) + '-diff'
    week_pct = 'week' + str(week) + '-pct'

    final_df[week_diff] = final_df[week_pred] - final_df[week_act]
    final_df[week_pct] = final_df[week_diff].div(final_df[week_pred].where(final_df[week_pred]!=0, np.nan))
    final_df[week_pred] = final_df[week_pred].astype(int)

    file_name = '/content/week' + str(week) + '-pred-offense-norookies-rb.csv'
    final_df.to_csv(file_name, index=False)

AR-1300
main function: player =  AR-1300
               n_periods =  16
               col =  85
lstm_pred function: series =  game
85.0      0.00
86.0      0.00
87.0      0.00
88.0      0.00
89.0      0.00
         ...  
316.0    28.12
317.0    11.40
318.0    14.42
319.0     9.34
320.0    19.02
Length: 236, dtype: float64
                    X =  [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   -2.6   0.   -0.1
  0.    0.    0.    0.56  0.    0.    0.    0.    0.    0.38  0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.58  0.    0.   15.04  0.    0.    0.    0.   20.62 25.62 18.6  10.4
 23.92 23.62 12.24 13.66  5.78 15.18 23.52 26.52 18.2  15.32 15.8  26.22
 15.06 18.74 28.56 21.96 20.22 26.14 28.68 23.24 19.06 24.06 24.42 21.52
  7.8  38.52 15.38 19.6  12.42 26.2  22.64 17.24 16.72 22.92 17.2   7.3
 27.66 31.14 26.86 29.82  2.34  0.   35.76 13.26 24.58 23.62 22.18 46.92
 24.74 25.14 27.2  31.08 28.1  25.76 21.18 31.96 17.24 22.6  34.12  0.
 2